In [1]:
import os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=False
)

In [4]:
# No tracing
agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)



> Entering new LLMMathChain chain...
36^2
Answer: 1296
> Finished LLMMathChain chain.


"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [12]:
# Stdout Tracing
os.environ["LANGCHAIN_TRACER"] = "stdout"

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)

Starting chain trace with inputs: {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"}, serialized: {'name': 'AgentExecutor'}, extra: {}
Starting chain trace with inputs: {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?", 'agent_scratchpad': '', 'stop': ['\nObservation: ']}, serialized: {'name': 'LLMChain'}, extra: {}
Starting LLM trace with prompts: ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [4]:
# Nested JSON tracing
os.environ["LANGCHAIN_TRACER"] = "nested_json"

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)



> Entering new LLMMathChain chain...
36^2
Answer: 1296
> Finished LLMMathChain chain.
{
  "id": "697ed67c-55e9-4611-92da-b20d0834e353",
  "start_time": 1671969681.334626,
  "end_time": 1671969689.239889,
  "extra": {},
  "error": null,
  "execution_order": 1,
  "serialized": {
    "name": "AgentExecutor"
  },
  "inputs": {
    "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
  },
  "outputs": {
    "output": "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."
  },
  "child_runs": [
    {
      "id": "ae8bf4b4-3120-4891-ae55-c433e2dcfe8a",
      "start_time": 1671969681.342222,
      "end_time": 1671969683.420435,
      "extra": {},
      "error": null,
      "execution_order": 2,
      "serialized": {
        "name": "LLMChain"
      },
      "inputs": {
        "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
   

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [4]:
# LangChain Tracing
os.environ["LANGCHAIN_TRACER"] = "langchain"

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)



> Entering new LLMMathChain chain...
36^2
Answer: 1296
> Finished LLMMathChain chain.
POST http://127.0.0.1:5000/chain-runs, status code: 201, id: 1


"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [5]:
import requests
r = requests.get("http://127.0.0.1:5000/chain-runs/1")
r.json()

{'id': 1,
 'start_time': 1671942136.535976,
 'end_time': 1671942144.259314,
 'extra': {},
 'error': None,
 'execution_order': 1,
 'serialized': {'name': 'AgentExecutor'},
 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
 'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
 'child_runs': [{'id': 4,
   'start_time': 1671942136.536082,
   'end_time': 1671942138.634476,
   'extra': {},
   'error': None,
   'execution_order': 2,
   'serialized': {'name': 'LLMChain'},
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    'agent_scratchpad': '',
    'stop': ['\nObservation: ']},
   'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "US Open men\'s ten

In [6]:
r = requests.get("http://127.0.0.1:5000/llm-runs")
r.json()

[{'id': 1,
  'start_time': '2022-12-24T20:22:22.127452',
  'end_time': '2022-12-24T20:22:22.739542',
  'extra': {},
  'error': None,
  'execution_order': 13,
  'serialized': {'name': 'OpenAI'},
  'prompts': {'prompts': ["You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```pyt

In [7]:
r = requests.get("http://127.0.0.1:5000/tool-runs")
r.json()

[{'id': 1,
  'start_time': '2022-12-24T20:22:18.636343',
  'end_time': '2022-12-24T20:22:18.703924',
  'extra': {},
  'error': None,
  'execution_order': 4,
  'serialized': {'name': '<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_en...'},
  'tool_input': "US Open men's tennis final 2019",
  'output': 'Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...',
  'action': 'Search',
  'child_runs': [],
  'child_llm_runs': [],
  'child_chain_runs': [],
  'child_tool_runs': []},
 {'id': 2,
  'start_time': '2022-12-24T20:22:20.181156',
  'end_time': '2022-12-24T20:22:20.698611',
  'extra': {},
  'error': None,
  'execution_order': 7,
  'serialized': {'name': '<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_en...'},
  'tool_input': 'Rafael Nadal age',
  'output': '36 years',
  'action': 'Search',
  'child_runs': [],
  'child_llm_runs': [],
  'child_chain_runs': [],
  'child_tool_runs': []},
 

In [12]:
r = requests.get("http://127.0.0.1:5000/chain-runs/top-level")
r.json()

[{'id': 1,
  'start_time': '2022-12-24T20:22:16.535976',
  'end_time': '2022-12-24T20:22:24.259314',
  'extra': {},
  'error': None,
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2022-12-24T20:22:16.536082',
    'end_time': '2022-12-24T20:22:18.634476',
    'extra': {},
    'error': None,
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the 

In [9]:
r = requests.get("http://127.0.0.1:5000/chain-runs")
r.json()

[{'id': 1,
  'start_time': '2022-12-24T20:22:16.535976',
  'end_time': '2022-12-24T20:22:24.259314',
  'extra': {},
  'error': None,
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2022-12-24T20:22:16.536082',
    'end_time': '2022-12-24T20:22:18.634476',
    'extra': {},
    'error': None,
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the 